In [18]:
genecodeR.flushdb()

True

In [35]:
genecodeDB = 4

# Set this to a large number (2**64) to 
geneNo = 100000

In [36]:
import redis
import json

In [37]:
genecodeR = redis.StrictRedis(host='localhost', db=genecodeDB)

In [38]:
genes = {}

def flushPreviousGene(chromosome, left, right, pieces):
    genecodeR.zadd("genecode$" + chromosome, left, json.dumps(pieces))    


with open("gencode.v25lift37.annotation.gtf") as geneList:
    currentGeneID = None
    currentGenePieces = []
    currentChromosome = None
    leftGene = 0
    rightGene = 0
    for i, line in enumerate(geneList):
        if i == geneNo:
            break
        if line and line[0] == "#":
            continue
        l = line.split(";")
        data = l[0].split("\t")
        transcriptID = l[1].split()[-1][1:-1]
        geneID = data[-1].split()[-1][1:-1]
        geneName = l[4].split()[-1][1:-1]
        typ = data[2]
        left = data[3]
        right = data[4]
        chromo = data[0]
        strand = data[6]
        if typ == "gene" and currentGeneID != geneID:
            if currentGeneID != None:
                flushPreviousGene(currentChromosome, leftGene, rightGene, currentGenePieces)
        if typ == "transcript":
            currentGeneID = transcriptID
            currentGenePieces = []
            currentChromosome = chromo
            leftGene = left
            rightGene = right
        unit = [typ, chromo, strand, left, right, geneID, geneName]
        currentGenePieces.append(unit)
    flushPreviousGene(currentChromosome, leftGene, rightGene, currentGenePieces)

In [39]:
c = "chr1"
left = 2439184
right = 2461794
response = []
for elem in genecodeR.zrangebyscore("genecode$" + c, left, right):
    parsedElem = json.loads(elem.decode("ascii", errors="ignore"))
    response.append(parsedElem)
response

[[['transcript',
   'chr1',
   '-',
   '2452806',
   '2458036',
   'ENSG00000157881.13_2',
   'PANK4'],
  ['exon', 'chr1', '-', '2457903', '2458036', 'ENSG00000157881.13_2', 'PANK4'],
  ['exon',
   'chr1',
   '-',
   '2452806',
   '2453239',
   'ENSG00000157881.13_2',
   'PANK4']],
 [['transcript',
   'chr1',
   '-',
   '2460184',
   '2461684',
   'ENSG00000197921.5_1',
   'HES5'],
  ['exon', 'chr1', '-', '2461550', '2461684', 'ENSG00000197921.5_1', 'HES5'],
  ['CDS', 'chr1', '-', '2461550', '2461603', 'ENSG00000197921.5_1', 'HES5'],
  ['start_codon',
   'chr1',
   '-',
   '2461601',
   '2461603',
   'ENSG00000197921.5_1',
   'HES5'],
  ['exon', 'chr1', '-', '2461285', '2461450', 'ENSG00000197921.5_1', 'HES5'],
  ['CDS', 'chr1', '-', '2461285', '2461450', 'ENSG00000197921.5_1', 'HES5'],
  ['exon', 'chr1', '-', '2460184', '2461188', 'ENSG00000197921.5_1', 'HES5'],
  ['CDS', 'chr1', '-', '2460911', '2461188', 'ENSG00000197921.5_1', 'HES5'],
  ['stop_codon',
   'chr1',
   '-',
   '2460908